In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Reshape,LSTM,Dropout,Activation,Bidirectional


In [161]:
df = pd.read_csv("CSV_Data_For_3.csv")

In [162]:
numpy_data = df.to_numpy()

In [163]:
sample_data = numpy_data[0:-1]


In [ ]:
0,0,1 -> 2
0,1,2 -> 3
1,2,3 -> 4
2,3,4 -> 5
3,4,5 -> 6

In [164]:
for val in sample_data:
    val[1] = np.array(eval(val[1]))
    val[2] = np.array(eval(val[2]))

In [167]:
result=[]
ORDER_SIZE = 3

x = []
y = []

two = np.zeros((2,2,3),dtype='object')


for (k, v) in order_food_dict.items():
    customer_orders = order_food_dict.get(k)
        
    if len(customer_orders) == 1:
        pass
    
    elif len(customer_orders) == 2:
        result.extend(two)
        result.extend(two)
        result.extend(customer_orders)
        
    elif len(customer_orders) == 3:
        result.extend(two)
        result.extend(customer_orders)
        
    else:
        result.extend(customer_orders)
        
for i in range(0, len(result)):
    if i + ORDER_SIZE < len(result):
        x.append(result[i : i+ORDER_SIZE])
        y.append(result[i+ORDER_SIZE][1])
        #print(f"Inputs -> {result[i : i+ORDER_SIZE]} ::::::: Output -> {result[i+ORDER_SIZE][1]} \n\n")
        

In [108]:
 # order_food_dict = {uniq: [[order, time], [order, time],   [order, time], [order, time]],   uniq2: [[order, time], [order, time],   [order, time], [order, time]]}

order_food_dict = {}
for sample in sample_data:
    key = sample[0]
    values = sample[1:]
    # check if the key is in the dictionary
    if order_food_dict.get(key) is not None:
         #   - get the value for that key
        existing_values = order_food_dict.get(key)
        # append new value to existing values
        existing_values.append(values)
        order_food_dict[key] = existing_values
        pass
    else:
        # create new values array
        new_values = []
        new_values.append(values)
        order_food_dict[key] = new_values
    
   
    pass

In [109]:
WINDOW_SIZE = 1
ORDER_SIZE = 3

new_dummy_data = []

for (k, v) in order_food_dict.items():
    customer_orders = order_food_dict.get(k)
    dummy_data = np.asarray(customer_orders)
    for j in range(len(dummy_data)):
        curr = list(dummy_data[j])
        if j + 1 < len(dummy_data):
            curr.append(dummy_data[j+1][1])
            pass
        else:
            curr.append([0,0,0])

        new_dummy_data.append(curr)
    

In [126]:
# new_dummy_data
x = []
y = []

In [111]:
for i in range(0, len(new_dummy_data)):
    if i + ORDER_SIZE < len(new_dummy_data):
        x.append(new_dummy_data[i : i+ORDER_SIZE])
        y.append(new_dummy_data[i+ORDER_SIZE][1])
        #print(f"Inputs -> {new_dummy_data[i : i+ORDER_SIZE]} ::::::: Output -> {new_dummy_data[i+ORDER_SIZE][1]} \n\n")

In [168]:
x_train= np.array(x,dtype='object')
y_train = np.array(y,dtype='object')

ValueError: could not broadcast input array from shape (2,3) into shape (2,)

In [45]:
inp_one = keras.layers.Input((ORDER_SIZE,3,3), name="Input_One")
inp_two = keras.layers.Input((ORDER_SIZE,3,3), name="Input_Two")

input_one = Dense(256, activation = 'relu', name="Dense_One")(inp_one)
input_two = Dense(128, activation = 'relu')(inp_two)

reshape_one = Reshape((ORDER_SIZE,256*3))(input_one)
reshape_two = Reshape((ORDER_SIZE,128*3))(input_two)

bi_lstm = Bidirectional(LSTM(50,return_sequences=True,activation='relu',dropout=0.2))(reshape_two)

lstm = LSTM(50,return_sequences=True,activation='relu',dropout=0.2)(reshape_one)


out = keras.layers.Concatenate()([bi_lstm, lstm])

flatten = keras.layers.Flatten()(out)

final = Dense(3,activation='sigmoid')(flatten)

model = keras.Model(inputs=[inp_one, inp_two], outputs= final)

In [46]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_Two (InputLayer)          [(None, 3, 3, 3)]    0                                            
__________________________________________________________________________________________________
Input_One (InputLayer)          [(None, 3, 3, 3)]    0                                            
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 3, 3, 128)    512         Input_Two[0][0]                  
__________________________________________________________________________________________________
Dense_One (Dense)               (None, 3, 3, 256)    1024        Input_One[0][0]                  
_______________________________________________________________________________________

In [14]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=20, monitor="val_acc"),
    tf.keras.callbacks.ModelCheckpoint(filepath='model.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

In [47]:
model.fit([x_train, x_train],y_train,epochs=50,batch_size=50,validation_split=0.2, callbacks=callbacks)

Epoch 1/50
131/131 [==============================] - 10s 73ms/step - loss: 0.3705 - acc: 0.8200 - val_loss: 0.1457 - val_acc: 0.9289
Epoch 2/50
131/131 [==============================] - 3s 25ms/step - loss: 0.1151 - acc: 0.9379 - val_loss: 0.1465 - val_acc: 0.9307
Epoch 3/50
131/131 [==============================] - 3s 24ms/step - loss: 0.1117 - acc: 0.9417 - val_loss: 0.1489 - val_acc: 0.9283
Epoch 4/50
131/131 [==============================] - 4s 34ms/step - loss: 0.1095 - acc: 0.9411 - val_loss: 0.1508 - val_acc: 0.9105
Epoch 5/50
131/131 [==============================] - 3s 27ms/step - loss: 0.1101 - acc: 0.9413 - val_loss: 0.1465 - val_acc: 0.9258
Epoch 6/50
131/131 [==============================] - 3s 25ms/step - loss: 0.1066 - acc: 0.9419 - val_loss: 0.1517 - val_acc: 0.9264
Epoch 7/50
131/131 [==============================] - 4s 27ms/step - loss: 0.1075 - acc: 0.9411 - val_loss: 0.1489 - val_acc: 0.9148
Epoch 8/50
131/131 [==============================] - 3s 24ms/step -

In [48]:
model.save("functional_1")

INFO:tensorflow:Assets written to: functional_1\assets


In [49]:
model_two = keras.models.load_model('functional_1')


In [1]:
test_inp = x_train[0].reshape(-1,ORDER_SIZE,3,3)

NameError: name 'x_train' is not defined

In [54]:
pred = model_two.predict([test_inp, test_inp])

In [55]:
np.round(pred)

array([[0., 0., 0.]], dtype=float32)